# PREPROCESSING DATA

In [138]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from functools import reduce

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [139]:
tweet = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

In [140]:
tweet[tweet.index == 389].text

389    Owner of Chicago-Area Gay Bar Admits to Arson ...
Name: text, dtype: object

1. Remove URL

In [141]:
def remove_URL(text):
#     # url = re.compile(r'https?://\S+|www\.\S+')
#     return re.sub(r'http\S+','', text)
    # url_pattern = re.compile(r'https?://\S+|www\.\S+')
    url_pattern = re.compile(r'\s*https?://\S+|www\.\S+\s*')
    return url_pattern.sub(r'', text)
check = "@bbcmtd ablaze http://t.co/lHYXEOHY6C ab"
check = remove_URL(check)
print(check)
print(len(check))

@bbcmtd ablaze ab
17


2. To_lower()

In [142]:
def to_lower(text):
  return text.lower()

3. Use contractions

In [143]:
!pip install contractions


In [144]:
import contractions
check = "I ain't haha because it's like english. I'll've @minh"
contractions.fix(check)

'I are not haha because it is like english. I will have @minh'

In [145]:
def use_contractions(text):
  return contractions.fix(text)

4. Remove number and punctuation

In [146]:
import string
string.punctuation, string.digits

('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~', '0123456789')

In [147]:
PUNCT_TO_REMOVE = str(string.punctuation + string.digits)

def remove_punc_and_num(text):
    """custom function to remove the punctuation"""
    for token in PUNCT_TO_REMOVE:
        text = text.replace(token, "")
    return text.strip()

check = "I'm am Minh, Minh is so so 2x abc @minh"
check = remove_punc_and_num(check)
print(check)
print(len(check))

Im am Minh Minh is so so x abc minh
35


5. Remove invalid char

In [148]:
def remove_invalid_char(text):
    return re.sub(r'[^a-zA-Z0-9\s]','',text)

check = remove_invalid_char(check)
print(check)
print(len(check))

Im am Minh Minh is so so x abc minh
35


6. Remove stopwords

In [149]:
with open('./dataset/stopwords.txt') as f:
    STOPWORDS = []
    for row in f:
        STOPWORDS.append(row.rstrip('\n'))

# Remove some words in stopwords
STOPWORDS.remove('no')
STOPWORDS.remove('not')

print(len(STOPWORDS))

171


In [150]:
def remove_stopwords(text):
    # stopwords_list = stopwords.words('english')
    return ' '.join([word for word in text.split() if word not in STOPWORDS])
check = "I'm am Minh, Minh is so so 2x abc"
check = remove_stopwords(check)
print(check)
print(len(check))

I'm Minh, Minh 2x abc
21


7. Emoticons and emojis

In [151]:
!pip install emot --upgrade
import emot
emot_obj = emot.emot()

In [152]:
#convert emoticons to words using emot
def handle_emoticons(text, remove_emoticon=False):
    dict_emoticons = dict(zip(emot_obj.emoticons(text)['value'], emot_obj.emoticons(text)['mean']))
    res_emoticons =  dict(sorted(dict_emoticons.items(), key = lambda kv:len(kv[1]), reverse=True))
    for emoticon, mean in res_emoticons.items():
        if remove_emoticon:
            text = text.replace(emoticon, "")
        else:
            text = text.replace(emoticon, mean)
    return text.strip()

def handle_emojis(text, remove_emoji=False):
    for emoji, mean in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
        if remove_emoji:
            text = text.replace(emoji, "")
        else:
            text = text.replace(emoji, mean.replace(":", ""))
    return text

check = "I :)) Troi oi la troi :3 arsonist :D 🤔 🤣"
check = handle_emoticons(check)
print(check)
print(len(check))

check = handle_emojis(check)
print(check)
print(len(check))

I Very Happy face or smiley Troi oi la troi Happy face smiley arsonist Laughing, big grin or laugh with glasses 🤔 🤣
115
I Very Happy face or smiley Troi oi la troi Happy face smiley arsonist Laughing, big grin or laugh with glasses thinking_face rolling_on_the_floor_laughing
155


9. Spelling Correction

In [153]:
!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

In [154]:
def correct_spelling(text):
    return spell(text)

print(correct_spelling("precessing"))
print(correct_spelling("helleo"))


processing
hello


10. Stemming and Lemmatization

In [155]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    for i in range(0,len(word_tokens)):
        word_tokens[i] = lemmatizer.lemmatize(word_tokens[i], 'v')
    return ' '.join([word for word in word_tokens])

print(lemmatize("running"))
print(lemmatize("I am a student"))

run
I be a student


Remove rare word

In [156]:
nltk.download('punkt')

from nltk import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Minh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [157]:
check = "I'm Minh Minh x abc alksdf"
check = word_tokenize(contractions.fix(check).lower())
print(check)

['i', 'am', 'minh', 'minh', 'x', 'abc', 'alksdf']


In [158]:

# def _get_mispell(mispell_dict):
#     mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
#     return mispell_dict, mispell_re

In [159]:
def preprocessing_csv(df):
    returned_df = df.copy()
    dict_func = [
        remove_URL,
        to_lower,
        use_contractions,
        remove_punc_and_num,
        remove_invalid_char,
        remove_stopwords,
        handle_emojis,
        handle_emoticons,
        correct_spelling,
        # remove_mention,
        lemmatize
    ]
    for func in dict_func:
        returned_df['text'] = returned_df['text'].apply(lambda x: func(x))

    returned_df['keyword'] = returned_df['keyword'].str.replace('%20', ' ')
    
    return returned_df

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def group_similar_texts_with_same_loc(df, threshold):
    # Convert the text data into numerical features using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df["text"])

    # Calculate the pairwise cosine similarity between documents
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Group similar documents with the given threshold and keep the row with the highest frequency of target
    groups = {}
    for i in range(len(df)):
        group_id = None
        for j in range(i):
            if cosine_sim[i,j] > threshold and df['location'][i] == df['location'][j]:
                if group_id is None:
                    group_id = j
                elif df["target"][j] > df["target"][group_id]:
                    group_id = j
        if group_id is None:
            group_id = i
        groups.setdefault(group_id, []).append(i)

    # Create a new DataFrame with the grouped data and the most common target in each group
    grouped_data = []
    for group in groups.values():
        target_freq = df.loc[group]["target"].value_counts()
        most_common_target = target_freq.index[0]
        representative = df.loc[(df["target"] == most_common_target) & (df.index.isin(group))].iloc[0]        
        grouped_data.append({
            "id": representative["id"],
            "keyword": representative["keyword"],
            "location": representative["location"],
            "text": ", ".join(df.loc[group]["text"]),
            "target": most_common_target
        })
    grouped_df = pd.DataFrame(grouped_data)

    return grouped_df

In [161]:
# df = preprocessing_csv(tweet)
# train_df = group_similar_texts_with_same_loc(df, 0.9)
# empty_text_rows = train_df[train_df['text'] == '']
# train_df = train_df.drop(empty_text_rows.index)
# train_df.to_csv('./preprocessing/train.csv', index = False)

In [162]:
df_test = preprocessing_csv(test)
df_test.to_csv('./preprocessing/test.csv', index = False)